## RAG 성능 테스트를 위한 함수 정의

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드|
load_dotenv()

True

In [2]:
import os

# LangSmith 추적 비활성화
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = ""

In [1]:
import json
import random

# JSON 파일 로드
json_path = "./json_data.json"  # JSON 파일 경로
with open(json_path, "r", encoding="utf-8") as f:
    artworks = json.load(f)

# 질문 템플릿
question_templates = {
    "year": "'{artist}'의 '{title}'은(는) 몇 년도에 제작되었나요?",
    "size": "'{artist}'의 '{title}' 크기는 어떻게 되나요?",
    "materials": "'{artist}'의 '{title}' 제작에 사용된 소재는 무엇인가요?",
    "category": "'{artist}'의 '{title}'은(는) 어떤 카테고리에 속하나요?",
    "artwork_number": "'{artist}'의 '{title}'의 작품 번호는 무엇인가요?"
}

# 평가 데이터 생성
qa_dataset = []
random.shuffle(artworks)  # 무작위로 섞기
selected_artworks = artworks[:100]  # 100개 선택

for artwork in selected_artworks:
    available_keys = [key for key in question_templates if key in artwork and artwork[key]]  # 값이 존재하는 필드만 사용
    if available_keys:
        selected_key = random.choice(available_keys)  # 랜덤하게 하나 선택
        question = question_templates[selected_key].format(artist=artwork['artist'], title=artwork['title'])
        answer = artwork[selected_key]

        qa_dataset.append({
            "text": f"<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response in Korean that appropriately completes the request.\n\n"
                    f"### Instruction:\n{question}\n\n"
                    f"### Input:\n\n\n"
                    f"### Response:\n{answer}<|endoftext|>"
        })

# JSON 파일 저장
json_output_path = "./qa_dataset.json"
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(qa_dataset, f, ensure_ascii=False, indent=4)

print(f"QA 데이터셋이 생성되었습니다: {json_output_path}")


QA 데이터셋이 생성되었습니다: ./qa_dataset.json


In [24]:
import json
import random

# JSON 파일 로드
json_path = "./json_data.json"  # JSON 파일 경로
with open(json_path, "r", encoding="utf-8") as f:
    artworks = json.load(f)

# 평가 데이터 생성
qa_dataset = []
random.shuffle(artworks)  # 무작위로 섞기
selected_artworks = artworks[:100]  # 100개 선택

for artwork in selected_artworks:
    if "artist" in artwork and "title" in artwork and "description" in artwork:
        question = f"'{artwork['artist']}'의 '{artwork['title']}'에 대해 설명해주세요."
        answer = artwork["description"]

        qa_dataset.append({
            "text": f"<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response in Korean that appropriately completes the request.\n\n"
                    f"### Instruction:\n{question}\n\n"
                    f"### Input:\n\n\n"
                    f"### Response:\n{answer}<|endoftext|>"
        })

# JSON 파일 저장
json_output_path = "./qa_dataset_description.json"
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(qa_dataset, f, ensure_ascii=False, indent=4)

print(f"QA 데이터셋이 생성되었습니다: {json_output_path}")


QA 데이터셋이 생성되었습니다: ./qa_dataset_description.json


In [2]:
import json
import random
from langsmith import Client
import pandas as pd

# data/qa_dataset.json 파일에서 데이터 불러오기
with open("./qa_dataset.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

# 데이터 100개 랜덤 추출
random.seed(42)  # 재현성을 위한 시드 설정
sampled_data = random.sample(data_list, 100)

# Langsmith 형식으로 변환 함수
def convert_to_langsmith_format(data_list):
    questions, answers = [], []
    for data in data_list:
        text = data["text"]
        
        # Instruction 추출
        instruction_start = text.find("### Instruction:\n") + len("### Instruction:\n")
        instruction_end = text.find("\n\n### Input:")
        question = text[instruction_start:instruction_end].strip()
        
        # Response 추출
        response_start = text.find("### Response:\n") + len("### Response:\n")
        response_end = text.find("<|endoftext|>")
        answer = text[response_start:response_end].strip()
        
        questions.append(question)
        answers.append(answer)
    return pd.DataFrame({"question": questions, "answer": answers})

# 변환 실행
df = convert_to_langsmith_format(sampled_data)

# Langsmith Client 연결
client = Client()
dataset_name = "RAG_EVAL_DATASET_NEW"

# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset

# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

print("Langsmith 데이터셋 100개 업로드가 완료되었습니다.")


Langsmith 데이터셋 100개 업로드가 완료되었습니다.


In [3]:
df.head()

,question,answer
0,'배만실'의 '태고의 흔적'은(는) 어떤 카테고리에 속하나요?,공예
1,'황규백'의 '당구'은(는) 어떤 카테고리에 속하나요?,판화
2,'손일봉'의 '정자'의 작품 번호는 무엇인가요?,2434
3,'임홍순'의 '고비' 크기는 어떻게 되나요?,120×30×6×(2)
4,'황규태'의 '픽셀'은(는) 어떤 카테고리에 속하나요?,사진


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:23<00:00,  3.40s/it]


In [5]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda
/tmp/ipykernel_839/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)


In [6]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks. 
Answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with "The document does not contain this information." 
Provide comprehensive answers, always include the artwork number, and ensure all answers are written in Korean. 
All answers should be formatted using beautiful Markdown syntax to make the response visually appealing and easy to read. 
Use headings, bullet points, and bold or italic text where appropriate to enrich the response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [7]:
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0114_docx"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [8]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 20,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.8      # 다양성과 관련성 간 균형
    }
)


In [9]:
import re
class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"### 모델 결과\n\n{extracted_text}\n\n"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"### 모델 결과\n\n{llm_output.strip()}\n\n"


In [10]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers.string import StrOutputParser
from langchain.prompts import ChatPromptTemplate
chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser()                    # 응답을 문자열로 변환
)


In [11]:
query = "노란저고리는 누구 작품인가요?"
response = chain.invoke({"question": query})
print(response)

### 모델 결과

**노란 저고리**는 **김종태** 작가의 작품입니다.  
**작품 번호**: 128  
**제작 연도**: 1929




In [12]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

In [13]:
# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "노란저고리는 누구 작품인가요?"}
)
llm_answer

{'answer': '### 모델 결과\n\n**노란 저고리**는 **김종태** 작가의 작품입니다. 작품 번호는 **PA-00128**입니다.\n\n'}

In [14]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

## Question-Answer Evaluator

In [14]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator("qa")

# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:


In [25]:
from langsmith import Client

client = Client()

# LangSmith에 존재하는 데이터셋 리스트 출력
datasets = list(client.list_datasets())

for dataset in datasets:
    print(f"Dataset Name: {dataset.name}, ID: {dataset.id}")


Dataset Name: RAG_EVAL_DATASET, ID: b2eb4069-1183-45d7-a21e-5ede0db30bca
Dataset Name: RAG_EVAL_DATASET_NEW, ID: 77cefd19-3314-4440-8ba8-f05ea51fb422


In [25]:
dataset_name = "RAG_EVAL_DATASET_NEW"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "QA Evaluator 를 활용한 평가 (1024)",
    },
)

View the evaluation results for experiment: 'RAG_EVAL-2da95c4c' at:
https://smith.langchain.com/o/a89b03f2-9920-4620-a0d1-5b700d444e04/datasets/d5f446c6-5b3c-47fd-a73e-d8d0c7079921/compare?selectedSessions=31a85e88-5669-4f38-b899-4eb854f5d13a




100it [06:56,  4.16s/it]


In [23]:
# LangSmith에서 데이터셋 불러오기
datasets = client.list_datasets()
for dataset in datasets:
    print(f"Dataset Name: {dataset.name}, ID: {dataset.id}")

# 데이터셋에서 데이터 샘플 확인 (제너레이터 → 리스트 변환)
examples = list(client.list_examples(dataset_id=dataset.id))  # 리스트로 변환

# 처음 5개만 출력
for example in examples[:5]:  
    print(example)


Dataset Name: RAG_EVAL_DATASET, ID: b2eb4069-1183-45d7-a21e-5ede0db30bca
Dataset Name: RAG_EVAL_DATASET_NEW, ID: 77cefd19-3314-4440-8ba8-f05ea51fb422
dataset_id=UUID('77cefd19-3314-4440-8ba8-f05ea51fb422') inputs={'question': "'김기승'의 '진신(전서)'에 대해 설명해주세요."} outputs={'answer': '원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955)까지 잇달아 서예부 특선을 차지하여 문교부장관상을 수상하였다. 1955년에는 대성서예원(大成書藝院)을 설립하였고, 1978년에는 원곡서예상(原谷書藝賞)을 제정하기도 하였다.김기승은 《제10회 대한민국미술전람회》(1961)의 취지문에서 "한국적 향기와 한국인의 체취를 풍기는 작품을 제작하기 위하여 온몸을 혹사하면서까지 많은 노력을 기울였으며, 서예의 경지를 어느 단계에 끌어올리려고 정성을 다하였다"라고 언급한 바 있다. 한국적 정취를 효과적으로 드러내면서도 특정 형식이나 글자 형태에 제한되지 않고 새롭고 율동적인 필세와 개성적인 감각을 추구하는 김기승의 작품들은 작가의 이러한 취지를 잘 드러낸다.또한 김기승은 원곡체(原谷體)를 만들어내고 묵영(墨映)을 창안하는 등, 서예계의 원로임에도 불구하고 새로움을 추구하는 데 게을리 하지 않은 작가이다. 원곡 자신이 전위적이라고 말하는 \'묵영\'이란 청묵(靑墨)의 번짐을 사용하거나 먹물의 농담을 이용하여 시각효과를 부각시킨 회화적 서예이다. 일부에서 묵영을 \'전통을 무시한 예술\'이라고 몰아붙이자 "전통을 지키기 위해서는 다각적인 실험작업을 통해 새로운 조형언어를 만들어내야 한다"고 맞설 정도로 원곡은 새로움을

## Heuristic Evaluation

In [26]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

In [27]:
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

# 토크나이저 선언
kiwi_tokenizer = KiwiTokenizer()

sent1 = "안녕하세요. 반갑습니다. 내 이름은 채림입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 채림입니다!!"

# 토큰화
print(sent1.split())
print(sent2.split())

print("===" * 20)

# 토큰화
print(kiwi_tokenizer.tokenize(sent1))
print(kiwi_tokenizer.tokenize(sent2))

['안녕하세요.', '반갑습니다.', '내', '이름은', '채림입니다.']
['안녕하세용', '반갑습니다~^^', '내', '이름은', '채림입니다!!']
['안녕', '하', '세요', '.', '반갑', '습니다', '.', '나', '의', '이름', '은', '채림', '이', 'ᆸ니다', '.']
['안녕', '하', '세요', 'ᆼ', '반갑', '습니다', '~', '^^', '나', '의', '이름', '은', '채림', '이', 'ᆸ니다', '!!']


In [31]:
from rouge_score import rouge_scorer

sent1 = "안녕하세요. 반갑습니다. 내 이름은 채림입니다."
sent2 = "안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!"
sent3 = "내 이름은 채림입니다. 안녕하세요. 반갑습니다."

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"], use_stemmer=False, tokenizer=KiwiTokenizer()
)

print(
    f"[1] {sent1}\n[2] {sent2}\n[rouge1] {scorer.score(sent1, sent2)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent2)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent2)['rougeL'].fmeasure:.5f}"
)
print("===" * 20)
print(
    f"[1] {sent1}\n[2] {sent3}\n[rouge1] {scorer.score(sent1, sent3)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent3)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent3)['rougeL'].fmeasure:.5f}"
)

[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!
[rouge1] 0.75862
[rouge2] 0.59259
[rougeL] 0.75862
[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 내 이름은 채림입니다. 안녕하세요. 반갑습니다.
[rouge1] 1.00000
[rouge2] 0.92857
[rougeL] 0.53333


In [32]:
from nltk.translate.bleu_score import sentence_bleu

sent1 = "안녕하세요. 반갑습니다. 내 이름은 채림입니다."
sent2 = "안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!"
sent3 = "내 이름은 채림입니다. 안녕하세요. 반갑습니다."

# 토큰화
print(kiwi_tokenizer.tokenize(sent1, type="sentence"))
print(kiwi_tokenizer.tokenize(sent2, type="sentence"))
print(kiwi_tokenizer.tokenize(sent3, type="sentence"))

안녕 하 세요 . 반갑 습니다 . 나 의 이름 은 채림 이 ᆸ니다 .
안녕 하 세여 반갑 습니다 ~~~ 나 의 이름 은 채림 이 ᆸ니다 !!
나 의 이름 은 채림 이 ᆸ니다 . 안녕 하 세요 . 반갑 습니다 .


In [33]:
bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent2, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent2}\n[score] {bleu_score:.5f}")
print("===" * 20)

bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent3, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {bleu_score:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!
[score] 0.75503
[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 내 이름은 채림입니다. 안녕하세요. 반갑습니다.
[score] 0.95739


In [36]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /home/chae/nltk_data...


True

In [37]:
from nltk.translate import meteor_score

sent1 = "안녕하세요. 반갑습니다. 내 이름은 채림입니다."
sent2 = "안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!"
sent3 = "내 이름은 채림입니다. 안녕하세요. 반갑습니다."

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent2, type="list"),
)

print(f"[1] {sent1}\n[2] {sent2}\n[score] {meteor:.5f}")
print("===" * 20)

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent3, type="list"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {meteor:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!
[score] 0.73077
[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 내 이름은 채림입니다. 안녕하세요. 반갑습니다.
[score] 0.96800


In [38]:
from sentence_transformers import SentenceTransformer, util
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

sent1 = "안녕하세요. 반갑습니다. 내 이름은 채림입니다."
sent2 = "안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!"
sent3 = "내 이름은 채림입니다. 안녕하세요. 반갑습니다."

# SentenceTransformer 모델 로드
model = SentenceTransformer("all-mpnet-base-v2")

# 문장들을 인코딩
sent1_encoded = model.encode(sent1, convert_to_tensor=True)
sent2_encoded = model.encode(sent2, convert_to_tensor=True)
sent3_encoded = model.encode(sent3, convert_to_tensor=True)

# sent1과 sent2 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent2_encoded).item()
print(f"[1] {sent1}\n[2] {sent2}\n[score] {cosine_similarity:.5f}")

print("===" * 20)

# sent1과 sent3 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent3_encoded).item()
print(f"[1] {sent1}\n[2] {sent3}\n[score] {cosine_similarity:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 안녕하세여 반갑습니다~~~ 내 이름은 채림입니다!!
[score] 0.88842
[1] 안녕하세요. 반갑습니다. 내 이름은 채림입니다.
[2] 내 이름은 채림입니다. 안녕하세요. 반갑습니다.
[score] 0.99265


In [39]:
from langsmith.schemas import Run, Example
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from sentence_transformers import SentenceTransformer, util
import os

# 토크나이저 병렬화 설정(HuggingFace 모델 사용)
os.environ["TOKENIZERS_PARALLELISM"] = "true"


def rouge_evaluator(metric: str = "rouge1") -> dict:
    # wrapper function 정의
    def _rouge_evaluator(run: Run, example: Example) -> dict:
        # 출력값과 정답 가져오기
        student_answer = run.outputs.get("answer", "")
        reference_answer = example.outputs.get("answer", "")

        # ROUGE 점수 계산
        scorer = rouge_scorer.RougeScorer(
            ["rouge1", "rouge2", "rougeL"], use_stemmer=True, tokenizer=KiwiTokenizer()
        )
        scores = scorer.score(reference_answer, student_answer)

        # ROUGE 점수 반환
        rouge = scores[metric].fmeasure

        return {"key": "ROUGE", "score": rouge}

    return _rouge_evaluator


def bleu_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="sentence")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="sentence")

    # BLEU 점수 계산
    bleu_score = sentence_bleu([reference_tokens], student_tokens)

    return {"key": "BLEU", "score": bleu_score}


def meteor_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="list")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="list")

    # METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    return {"key": "METEOR", "score": meteor}


def semscore_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # SentenceTransformer 모델 로드
    model = SentenceTransformer("all-mpnet-base-v2")

    # 문장 임베딩 생성
    student_embedding = model.encode(student_answer, convert_to_tensor=True)
    reference_embedding = model.encode(reference_answer, convert_to_tensor=True)

    # 코사인 유사도 계산
    cosine_similarity = util.pytorch_cos_sim(
        student_embedding, reference_embedding
    ).item()

    return {"key": "sem_score", "score": cosine_similarity}

In [61]:
from langsmith.evaluation import evaluate

# 평가자 정의
heuristic_evalulators = [
    rouge_evaluator(metric="rougeL"),
    bleu_evaluator,
    meteor_evaluator,
    semscore_evaluator,
]

# 데이터셋 이름 설정
dataset_name = "ds-0204"

# 실험 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=heuristic_evalulators,
    experiment_prefix="Heuristic-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Heuristic-EVAL (Rouge, BLEU, METEOR, SemScore) 을 사용하여 평가",
    },
)

View the evaluation results for experiment: 'Heuristic-EVAL-c226f7a4' at:
https://smith.langchain.com/o/a89b03f2-9920-4620-a0d1-5b700d444e04/datasets/5aae75ae-47e4-48e5-823c-eb233a86f9b9/compare?selectedSessions=fe8da49f-37f6-4c33-b673-2f14fff635db




10it [06:14, 37.46s/it]


### Langsmith를 활용하지 않은 로컬 평가

In [23]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate import meteor_score
from kiwipiepy import Kiwi


# 모델 및 토크나이저 초기화
model = SentenceTransformer("all-mpnet-base-v2")
kiwi = Kiwi()

csv_file_path = 'dataset_csv/gpt_qa_translate.csv'
df = pd.read_csv(csv_file_path)

results = []

for i, row in df.iterrows():
    question = str(row['question'].strip())
    ground_truth = str(row['ground_truth']).strip()

    # ✅ RAG 모델을 이용해 답변 생성
    model_answer = ask_question({"question": question})
    model_text = model_answer["answer"]  # ✅ 중요!

    # ✅ SemScore 계산
    ground_embedding = model.encode(ground_truth, convert_to_tensor=True)
    model_embedding = model.encode(model_text, convert_to_tensor=True)
    sem_score = util.pytorch_cos_sim(model_embedding, ground_embedding).item()

    # ✅ ROUGE 점수 계산 (ROUGE-L 제거)
    rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2"], use_stemmer=True)
    rouge_scores = rouge.score(ground_truth, model_text)
    rouge1 = rouge_scores["rouge1"].fmeasure
    rouge2 = rouge_scores["rouge2"].fmeasure

    # ✅ BLEU 점수 계산 (Smoothing 추가)
    reference_tokens = [token.form for token in kiwi.tokenize(ground_truth)]
    student_tokens = [token.form for token in kiwi.tokenize(model_text)]
    smoothing = SmoothingFunction().method1  # BLEU 스무딩 적용
    bleu_score = sentence_bleu([reference_tokens], student_tokens, smoothing_function=smoothing)

    # ✅ METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    # ✅ 결과 저장
    results.append([i + 1, question, model_text, ground_truth, rouge1, rouge2, bleu_score, meteor, sem_score])

    print(f"✅ {i+1}/{len(df)} 질문 완료: {question} -> SemScore: {sem_score:.4f}, ROUGE-1: {rouge1:.4f}, ROUGE-2: {rouge2:.4f}, BLEU: {bleu_score:.4f}, METEOR: {meteor:.4f}")

# ✅ CSV 저장
csv_output = "evaluation_results_0206.csv"
df_results = pd.DataFrame(results, columns=[
    "Index", "Question", "RAG Answer", "Ground Truth",
    "ROUGE-1", "ROUGE-2", "BLEU", "METEOR", "SemScore"
])
df_results.to_csv(csv_output, index=False, encoding="utf-8")

print(f"✅ 평가 완료! 결과가 '{csv_output}' 파일에 저장되었습니다.")


✅ 1/10 질문 완료: 한국 추상회화의 독특한 접근 방식으로 잘 알려진 박항섭의 작품은 특히 개인적, 역사적 주제에 대한 탐구라는 측면에서 그의 독특한 세계관을 어떻게 반영하고 있나요? -> SemScore: 0.7849, ROUGE-1: 0.6667, ROUGE-2: 0.0000, BLEU: 0.0776, METEOR: 0.3715
✅ 2/10 질문 완료: 홍정희 작가의 작품 '광산-희망'에 사용된 콜라주 기법은 어떻게 사용되었나요? -> SemScore: 0.9507, ROUGE-1: 0.0000, ROUGE-2: 0.0000, BLEU: 0.0731, METEOR: 0.3773
✅ 3/10 질문 완료: 최종태 작가의 '생각 속의 여인'의 특징과 화풍은 무엇인가요? -> SemScore: 0.7896, ROUGE-1: 0.0000, ROUGE-2: 0.0000, BLEU: 0.0580, METEOR: 0.3217
✅ 4/10 질문 완료: 박항섭과 김경원의 작품에서 볼 수 있는 한국 추상회화의 특징은 무엇인가요? -> SemScore: 0.9314, ROUGE-1: 0.0000, ROUGE-2: 0.0000, BLEU: 0.0650, METEOR: 0.3387
✅ 5/10 질문 완료: 한국의 풍경과 가족을 강조한 1957년 이수옥의 작품으로 여인과 두 아이가 등장하는 작품은 무엇인가요? -> SemScore: 0.8470, ROUGE-1: 0.4000, ROUGE-2: 0.0000, BLEU: 0.0295, METEOR: 0.3886
✅ 6/10 질문 완료: 박성환의 '군무'(1976)는 한국의 풍경과 개인의 변화를 어떻게 묘사하고 있을까요? -> SemScore: 0.8926, ROUGE-1: 0.4000, ROUGE-2: 0.0000, BLEU: 0.0666, METEOR: 0.4056
✅ 7/10 질문 완료: 장완 감독의 '침묵'에서 색과 빛은 어떻게 사실적인 단순함과 역동적인 표현의 매력적인 분위기를 불러일으킬 수 있을까요? -> SemScore: 

In [25]:
import json
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate import meteor_score
from kiwipiepy import Kiwi


# ✅ 모델 및 토크나이저 초기화
model = SentenceTransformer("all-mpnet-base-v2")
kiwi = Kiwi()

# ✅ JSON 파일 로드
json_file_path = "qa_dataset_description.json"  # JSON 파일 경로
with open(json_file_path, "r", encoding="utf-8") as f:
    qa_data = json.load(f)

# ✅ JSON 데이터를 DataFrame으로 변환
qa_pairs = []
for item in qa_data:
    text_parts = item["text"].split("### Instruction:\n")  # 질문 추출
    if len(text_parts) > 1:
        question_part = text_parts[1].split("\n\n### Input:\n\n\n")[0]  # 질문
        answer_part = text_parts[1].split("### Response:\n")[1].replace("<|endoftext|>", "").strip()  # 정답
        qa_pairs.append({"question": question_part, "ground_truth": answer_part})

df = pd.DataFrame(qa_pairs)

# ✅ 평가 진행
results = []

for i, row in df.iterrows():
    question = str(row["question"].strip())
    ground_truth = str(row["ground_truth"]).strip()

    # ✅ RAG 모델을 이용해 답변 생성
    model_answer = ask_question({"question": question})
    model_text = model_answer["answer"]  # ✅ 중요!

    # ✅ SemScore 계산
    ground_embedding = model.encode(ground_truth, convert_to_tensor=True)
    model_embedding = model.encode(model_text, convert_to_tensor=True)
    sem_score = util.pytorch_cos_sim(model_embedding, ground_embedding).item()

    # ✅ ROUGE 점수 계산 (ROUGE-L 제거)
    rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2"], use_stemmer=True)
    rouge_scores = rouge.score(ground_truth, model_text)
    rouge1 = rouge_scores["rouge1"].fmeasure
    rouge2 = rouge_scores["rouge2"].fmeasure

    # ✅ BLEU 점수 계산 (Smoothing 추가)
    reference_tokens = [token.form for token in kiwi.tokenize(ground_truth)]
    student_tokens = [token.form for token in kiwi.tokenize(model_text)]
    smoothing = SmoothingFunction().method1  # BLEU 스무딩 적용
    bleu_score = sentence_bleu([reference_tokens], student_tokens, smoothing_function=smoothing)

    # ✅ METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    # ✅ 결과 저장
    results.append([i + 1, question, model_text, ground_truth, rouge1, rouge2, bleu_score, meteor, sem_score])

    print(f"✅ {i+1}/{len(df)} 질문 완료: {question} -> SemScore: {sem_score:.4f}, ROUGE-1: {rouge1:.4f}, ROUGE-2: {rouge2:.4f}, BLEU: {bleu_score:.4f}, METEOR: {meteor:.4f}")

# ✅ CSV 저장
csv_output = "dataset_csv/evaluation_results_description_0206.csv"
df_results = pd.DataFrame(results, columns=[
    "Index", "Question", "RAG Answer", "Ground Truth",
    "ROUGE-1", "ROUGE-2", "BLEU", "METEOR", "SemScore"
])
df_results.to_csv(csv_output, index=False, encoding="utf-8")

print(f"✅ 평가 완료! 결과가 '{csv_output}' 파일에 저장되었습니다.")


✅ 1/100 질문 완료: '나혜석'의 '화령전작약(華寧殿芍藥)'에 대해 설명해주세요. -> SemScore: 0.7187, ROUGE-1: 0.1538, ROUGE-2: 0.0000, BLEU: 0.1354, METEOR: 0.2207
✅ 2/100 질문 완료: '정희승'의 '무제'에 대해 설명해주세요. -> SemScore: 0.6665, ROUGE-1: 0.0000, ROUGE-2: 0.0000, BLEU: 0.0845, METEOR: 0.2142
✅ 3/100 질문 완료: '한운성'의 'Ranunculus Asiaticus'에 대해 설명해주세요. -> SemScore: 0.4556, ROUGE-1: 0.1176, ROUGE-2: 0.0000, BLEU: 0.1451, METEOR: 0.2758
✅ 4/100 질문 완료: '홍선웅'의 '민중판화 대표작품'에 대해 설명해주세요. -> SemScore: 0.7460, ROUGE-1: 0.1176, ROUGE-2: 0.0000, BLEU: 0.2435, METEOR: 0.4598
✅ 5/100 질문 완료: '유영국'의 '작품005'에 대해 설명해주세요. -> SemScore: 0.5674, ROUGE-1: 0.1739, ROUGE-2: 0.0909, BLEU: 0.1844, METEOR: 0.3270
✅ 6/100 질문 완료: '김차섭'의 '자화상'에 대해 설명해주세요. -> SemScore: 0.6968, ROUGE-1: 0.1053, ROUGE-2: 0.0000, BLEU: 0.0711, METEOR: 0.2717
✅ 7/100 질문 완료: '아우구스토 바로스'의 '연을 가진 남자'에 대해 설명해주세요. -> SemScore: 0.6347, ROUGE-1: 0.5161, ROUGE-2: 0.3448, BLEU: 0.1340, METEOR: 0.4950
✅ 8/100 질문 완료: '하태진'의 '설경'에 대해 설명해주세요. -> SemScore: 0.5843, ROUGE-1: 0.1333, ROUGE-2: 0.